## Method 1 : By including adversarial examples in the training data.

In [1]:
import tensorflow as tf

from tensorflow.keras.datasets import mnist

from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation

import numpy as np
import random

import matplotlib.pyplot as plt

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
labels = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']

In [4]:
img_rows, img_cols, channels = 28, 28, 1
num_classes = 10

x_train = x_train / 255
x_test = x_test / 255

x_train = x_train.reshape((-1, img_rows, img_cols, channels))
x_test = x_test.reshape((-1, img_rows, img_cols, channels))

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

print("Data shapes", x_test.shape, y_test.shape, x_train.shape, y_train.shape)

Data shapes (10000, 28, 28, 1) (10000, 10) (60000, 28, 28, 1) (60000, 10)


In [5]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), strides=(3, 3), padding='same', activation='relu', input_shape=(img_rows, img_cols, channels)))
    model.add(Conv2D(64, kernel_size=(3, 3), strides=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), strides=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(32))
    model.add(Dropout(0.2))
    model.add(Dense(32))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    
    return model

model = create_model()

In [6]:
model.fit(x_train, y_train,
          batch_size=32,
          epochs=20,
          validation_data=(x_test, y_test))

Epoch 1/20
1875/1875 [==============================] - 17s 8ms/step - loss: 0.0333 - accuracy: 0.7506 - val_loss: 0.0067 - val_accuracy: 0.9568
Epoch 2/20
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0100 - accuracy: 0.9347 - val_loss: 0.0051 - val_accuracy: 0.9660
Epoch 3/20
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0076 - accuracy: 0.9512 - val_loss: 0.0046 - val_accuracy: 0.9702
Epoch 4/20
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0067 - accuracy: 0.9567 - val_loss: 0.0039 - val_accuracy: 0.9748
Epoch 5/20
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0057 - accuracy: 0.9635 - val_loss: 0.0036 - val_accuracy: 0.9773
Epoch 6/20
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0051 - accuracy: 0.9682 - val_loss: 0.0040 - val_accuracy: 0.9744
Epoch 7/20
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0052 - accuracy: 0.9666 - val_loss: 0.0036 - val_ac

In [7]:
print("Base accuracy on regular images:", model.evaluate(x=x_test, y=y_test, verbose=0))

Base accuracy on regular images: [0.003271447727456689, 0.9794999957084656]


In [8]:
def adversarial_pattern(image, label):
    image = tf.cast(image, tf.float32)
    
    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.MSE(label, prediction)
    
    gradient = tape.gradient(loss, image)
    
    signed_grad = tf.sign(gradient)
    
    return signed_grad

In [10]:
def generate_adversarials(batch_size):
    while True:
        x = []
        y = []
        for batch in range(batch_size):
            N = random.randint(0, 100)

            label = y_train[N]
            image = x_train[N]
            
            perturbations = adversarial_pattern(image.reshape((1, img_rows, img_cols, channels)), label).numpy()
            
            
            epsilon = 0.1
            adversarial = image + perturbations * epsilon
            
            x.append(adversarial)
            y.append(y_train[N])
        
        
        x = np.asarray(x).reshape((batch_size, img_rows, img_cols, channels))
        y = np.asarray(y)
        
        yield x, y

In [11]:
# Generate an adversarial test dataset:
x_adversarial_test, y_adversarial_test = next(generate_adversarials(10000)) 
print("Base accuracy on adversarial images:", model.evaluate(x=x_adversarial_test, y=y_adversarial_test, verbose=0))


Base accuracy on adversarial images: [0.10761824995279312, 0.3619000017642975]


# Training a defended model <br>
Training the model on a generator allows new images to be made to fool the model, which would mean that the model would learn to adapt to more complex images, and become more robust in the process.

In [12]:
x_adversarial_train, y_adversarial_train = next(generate_adversarials(20000))


In [13]:
model.fit(x_adversarial_train, y_adversarial_train,
          batch_size=32,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
625/625 [==============================] - 5s 9ms/step - loss: 0.0063 - accuracy: 0.9655 - val_loss: 0.0088 - val_accuracy: 0.9515
Epoch 2/10
625/625 [==============================] - 6s 9ms/step - loss: 0.0014 - accuracy: 0.9929 - val_loss: 0.0091 - val_accuracy: 0.9492
Epoch 3/10
625/625 [==============================] - 5s 9ms/step - loss: 1.2791e-04 - accuracy: 0.9992 - val_loss: 0.0100 - val_accuracy: 0.9441
Epoch 4/10
625/625 [==============================] - 5s 9ms/step - loss: 1.2594e-04 - accuracy: 0.9991 - val_loss: 0.0110 - val_accuracy: 0.9394
Epoch 5/10
625/625 [==============================] - 5s 8ms/step - loss: 2.0336e-04 - accuracy: 0.9987 - val_loss: 0.0115 - val_accuracy: 0.9375
Epoch 6/10
625/625 [==============================] - 5s 9ms/step - loss: 1.5029e-04 - accuracy: 0.9991 - val_loss: 0.0108 - val_accuracy: 0.9401
Epoch 7/10
625/625 [==============================] - 5s 9ms/step - loss: 1.0939e-04 - accuracy: 0.9994 - val_loss: 0.0116 - val_acc

In [16]:
# Although the model can defeat the images that used to fool it, we can still generate new images based on its weaknesses
x_adversarial_test, y_adversarial_test = next(generate_adversarials(10000))


In [17]:
print("Defended accuracy on adversarial images:", model.evaluate(x=x_adversarial_test, y=y_adversarial_test, verbose=0))
print("Defended accuracy on regular images:", model.evaluate(x=x_test, y=y_test, verbose=0))

Defended accuracy on adversarial images: [0.01784554496407509, 0.9111999869346619]
Defended accuracy on regular images: [0.01233533676713705, 0.9336000084877014]
